In [0]:
# 1. Get list of unique states from your Gold Table
# We collect this dynamically so the dropdown is always up to date
states_list = [row['state'] for row in spark.sql("SELECT DISTINCT state FROM agriculture.gold.roi_recommendations ORDER BY state").collect()]

# 2. Create the Dropdown Widget
dbutils.widgets.dropdown("selected_state", states_list[0], states_list, "Select Region")

# 3. Create a "View Mode" Widget
dbutils.widgets.dropdown("view_mode", "Profitability", ["Profitability", "Stability (Low Risk)"], "Sort By")

print("UI Widgets created. Look at the top of the notebook")

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, desc, asc, row_number

# 1. Capture User Inputs
user_region = dbutils.widgets.get("selected_state")
user_mode = dbutils.widgets.get("view_mode")

print(f"ANALYZING MARKET DATA FOR: {user_region}")
print(f"OPTIMIZING FOR: {user_mode}")

# 2. Base Filter
df_gold = spark.table("agriculture.gold.roi_recommendations") \
    .filter(col("state") == user_region) \
    .filter(col("forecast_date") >= current_date())

# 3. Define Sorting Strategy
if user_mode == "Profitability":
    sort_col = desc("estimated_revenue")
else:
    sort_col = asc("risk_factor")

windowSpec = Window.partitionBy("crop_name").orderBy(sort_col)

df_diverse = df_gold.withColumn("rank", row_number().over(windowSpec)) \
    .filter(col("rank") == 1) \
    .orderBy(sort_col) 

display(df_diverse.select(
    "crop_name", 
    "district", 
    "forecast_date", 
    "estimated_revenue", 
    "predicted_price", 
    "risk_category"
))

Databricks visualization. Run in Databricks to view.